In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings
warnings.filterwarnings('ignore')
random.seed(1)

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [2]:
example = pd.read_csv('example.csv')

In [270]:
train_df, test_df = train_test_split(example, train_size = 0.8)
X_train = train_df.drop(['Unnamed: 0', 'Grant.Status'], axis=1)
y_train = train_df['Grant.Status']
X_test = test_df.drop(['Unnamed: 0', 'Grant.Status'], axis=1)
y_test = test_df['Grant.Status']

In [271]:
gbclf = GradientBoostingClassifier(n_estimators=70, max_depth=4, verbose=0)
gbclf.fit(X_train,y_train)
X_train_leaves = gbclf.apply(X_train)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbclf.fit(X_test,y_test)
X_test_leaves = gbclf.apply(X_test)[:,:,0]

In [268]:
lr = LogisticRegression()
lr.fit(X_train_leaves, y_train)
y_pred_gbdtlr1 = lr.predict_proba(X_test_leaves)[:,1]
gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)

gbdt+lr auc 1: 0.72128


In [272]:
print(len(X_train_leaves))
print(len(X_train_leaves[0]))
y_pred_gbdtlr1 = lr.predict_proba(X_test_leaves)[:,1]
gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)

5794
70
gbdt+lr auc 1: 0.26255


In [273]:
gbc=GradientBoostingClassifier()
gbc.fit(X_train,y_train)
y_pred=gbc.predict_proba(X_test)[:,1]
auc=roc_auc_score(y_test,y_pred)
print(auc)

0.935327385724


In [274]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_pred_xgb=xgb.predict_proba(X_test)[:,1]
auc=roc_auc_score(y_test,y_pred_xgb)
print(auc)

0.935067798163


In [135]:
gnb= GaussianNB()
gnb.fit(X_train_leaves, y_train)
Y_pred_nb=gnb.predict_proba(X_test_leaves)[:,1]
gnb_auc = roc_auc_score(y_test,Y_pred_nb)
print('NB auc: ', gnb_auc)

NB auc:  0.657571872229


In [136]:
svc=SVC(probability=True)

svc.fit(X_train_leaves, y_train)
Y_pred_svc=svc.predict_proba(X_test_leaves)[:,1]
svc_auc=roc_auc_score(y_test,Y_pred_svc)
print('SVC auc: %.5f' % svc_auc)

SVC auc: 0.65157


In [137]:
knn=KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train_leaves, y_train)
Y_pred_knn=knn.predict_proba(X_test_leaves)[:,1]
knn_auc=roc_auc_score(y_test,Y_pred_knn)
print('KNN auc : %.5f' % knn_auc)

KNN auc : 0.80715


In [138]:
#perceptron
perceptron = Perceptron()
perceptron.fit(X_train_leaves, y_train)
Y_pred_prec=perceptron.predict(X_test_leaves)
perc_auc=roc_auc_score(y_test,Y_pred_prec)
print('Perceptron auc : ', perc_auc)

Perceptron auc :  0.633917089256


In [139]:
#linear svc
lin = LinearSVC()
lin.fit(X_train_leaves, y_train)
Y_pred_lin=lin.predict(X_test_leaves)
lin_auc=roc_auc_score(y_test,Y_pred_lin)
print('Linear SVC auc : ', lin_auc)

Linear SVC auc :  0.51237250333


In [140]:
#SGD
sgd = SGDClassifier(loss='log')
sgd.fit(X_train_leaves, y_train)
Y_pred_sgd=sgd.predict_proba(X_test_leaves)[:, 1]
sgd_auc=roc_auc_score(y_test,Y_pred_sgd)
print('SGD auc : %.5f' % sgd_auc)

SGD auc : 0.73946


In [141]:
#XGB

xgb=XGBClassifier()
xgb.fit(X_train_leaves,y_train)
Y_pred_xgb=xgb.predict(X_test_leaves)
xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
print(xgb_auc)

0.633002491391


In [142]:
lr = LogisticRegression(n_jobs=-1)
X_train_ext = hstack([X_train_leaves, X_train])
lr.fit(X_train_ext, y_train)
X_test_ext = hstack([X_test_leaves, X_test])
y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
print('gbdt+lr auc 2: %.5f' % gbdtlr_auc2)

gbdt+lr auc 2: 0.80278


In [275]:
te=pd.read_csv('testing.csv')

In [276]:
te.head()

,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,Start.date,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,...,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1
0,NaN,47.0,2.0,1.0,2009-01-01,NaN,NaN,NaN,NaN,NaN,...,2603.0,25.0,1.0,3.0,4.0,2.0,8.0,4.0,10.0,0.0
1,NaN,32.0,3.0,7.0,2009-01-01,270199.0,100.0,0.0,0.0,0.0,...,2828.0,25.0,0.0,3.0,5.0,3.0,4.0,5.0,5.0,0.0
2,NaN,99.0,3.0,10.0,2009-01-01,NaN,NaN,NaN,NaN,NaN,...,2853.0,25.0,1.0,5.0,0.0,2.0,9.0,34.0,22.0,20.0
3,NaN,2.0,1.0,8.0,2009-01-01,280204.0,60.0,280506.0,20.0,230119.0,...,2153.0,19.0,1.0,3.0,4.0,0.0,5.0,3.0,6.0,2.0
4,NaN,24.0,3.0,2.0,2009-01-01,320502.0,50.0,320703.0,50.0,0.0,...,2668.0,25.0,1.0,4.0,2.0,7.0,11.0,3.0,2.0,0.0


In [281]:
X_test1=te.drop(['Grant.Status','Start.date'], axis=1)

In [279]:
y_test1=te['Grant.Status']

In [286]:
y_pred_te=xgb.predict(X_test1)

In [287]:
y_pred_te

array([1, 0, 0, ..., 1, 1, 0])

In [288]:
test_df = pd.read_csv('/Users/garb47/2018HCIntern/datasets/unimelb_test.csv')

In [289]:
submission = pd.DataFrame({
    'Grant.Application.ID': test_df['Grant.Application.ID'],
    'Survived': y_pred_te
})
submission.to_csv('grant.csv', index=False)